In [65]:
import requests     
import json         
import pandas as pd 
import time

In [2]:
path = "./Data/withEOD.csv"

df = pd.read_csv(path)  
df

,Code-EOD,Country-EOD,Currency-EOD,Exchange-EOD,ISIN,Name-EOD,Type-EOD,previousClose-EOD,previousCloseDate-EOD,country-Inv,...,symbol-Inv,asset_class-Inv,Name_197,industry-IB,longName-IB,marketName-IB,Symbol-IB,Country-IB,Exchange-IB,Currency-IB
0,TWOU,USA,USD,US,US90214J1016,2U Inc,Common Stock,9.200,2022-08-02,united states,...,TWOU,equity,2U INC,"Consumer, Non-cyclical",2U INC,NMS,TWOU,NaN,NaN,NaN
1,88E,UK,GBX,LSE,AU00000088E2,88 Energy Ltd,Common Stock,0.625,2022-08-03,united kingdom,...,88E,equity,88 ENERGY LTD,Energy,88 ENERGY LTD,88E,88E,United Kingdom,LSE,GBP
2,7063,Japan,JPY,TSE,JP3160590000,A Dot Co Ltd,Common Stock,2064.000,2022-08-03,japan,...,7063,equity,A DOT CO LTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,Hong Kong,HKD,HK,KYG2953R1149,AAC Technologies Holdings Inc,Common Stock,14.580,2022-08-03,hong kong,...,2018,equity,AAC TECHNOLOGIES HOLDINGS IN,Industrial,AAC TECHNOLOGIES HOLDINGS IN,2018,2018,NaN,NaN,NaN
4,2686,Hong Kong,HKD,HK,KYG000371040,AAG Energy Holdings Ltd,Common Stock,1.410,2022-08-03,hong kong,...,2686,equity,AAG ENERGY HOLDINGS LTD,Energy,AAG ENERGY HOLDINGS LTD,2686,2686,Hong Kong,SEHK,HKD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17506,NaN,NaN,NaN,NaN,US98979J1097,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ZOE'S KITCHEN INC,"Consumer, Cyclical",ZOE'S KITCHEN INC,ZOES,ZOES,NaN,NaN,NaN
17507,NaN,NaN,NaN,NaN,CA98977K1084,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ZOLOTO RESOURCES LTD,Basic Materials,ZOLOTO RESOURCES LTD,ZR,ZR,NaN,NaN,NaN
17508,NaN,NaN,NaN,NaN,CA98977R1038,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ZOOMMED INC,Technology,ZOOMMED INC,ZMD.H,ZMD.H,NaN,NaN,NaN
17509,NaN,NaN,NaN,NaN,GB00BMHTHT14,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ZPG PLC,Communications,ZPG PLC,ZPG,ZPG,NaN,NaN,NaN


In [13]:
len(ticker_list)

100

In [94]:
# key & urls
# openfigi_apikey = '71aaa940-3567-4040-bce5-9f43490a7703' # GAI
openfigi_apikey = '6531f095-b68a-44c2-9749-20b3b75f9ad8'
openfigi_url = 'https://api.openfigi.com/v2/mapping'
openfigi_headers = {'Content-Type': 'text/json'}

In [21]:
def map_jobs(jobs):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, 
                             headers=openfigi_headers,
                             json=jobs)
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return response.json()

In [95]:
def get100Figi(ticker_list):
    
    jobs = []
    for i in ticker_list:
        s = "{'idType': 'ID_ISIN', 'idValue': '" + str(i) + "'}"
        json_acceptable_string = s.replace("'", "\"")
        d = json.loads(json_acceptable_string)
        jobs.append(d)
    
    
    job_results = map_jobs(jobs)
    
    # Extract figi from scrape results
    figi_value = [] # store figi information
    figi_count = [] # keep track of the number of each symbol's corresponding figi
    j=0
    for d in job_results:
        if 'data' in d:
            figi_count.append(len(d['data']))
            for i in range(len(d['data'])):   
                figi_value.append(d['data'][i])
                d['data'][i]['ISIN'] = ticker_list[j]
        else:
#             d['data'][i]['ISIN'] = ticker_list[j]
            figi_count.append(0)
        j+=1
        
    df_figi = pd.DataFrame.from_dict(figi_value)
    
    # extract useful information and rename
    df_res = df_figi[['ISIN', 'name', 'ticker', 'figi', 'compositeFIGI', 'shareClassFIGI', 'exchCode', 'marketSector', 'securityDescription', 'securityType', 'securityType2']].rename(columns = {'name':'longName', 'ticker':'symbol'})
    
    return df_res


In [96]:
len(df)//100 + 1

176

In [97]:
requestTimes = len(df)//100 + 1
df_figi = pd.DataFrame()

for i in range(requestTimes):
    # get 100 tickers
    ticker_list = list(df['ISIN'].unique())[100*(i):100*(i+1)]
    # get figi
    df100 = get100Figi(ticker_list)
    df100 = df100.drop_duplicates(subset=['ISIN'])
    df_figi = pd.concat([df_figi, df100])
#     time.sleep()
    
    # print progress
    if i % (requestTimes//10) == 0:
        print("progress: %.2f" % (100*i/requestTimes) + "%")

df_figi

progress: 0.00%
progress: 9.66%
progress: 19.32%
progress: 28.98%
progress: 38.64%
progress: 48.30%
progress: 57.95%
progress: 67.61%
progress: 77.27%
progress: 86.93%
progress: 96.59%


,ISIN,longName,symbol,figi,compositeFIGI,shareClassFIGI,exchCode,marketSector,securityDescription,securityType,securityType2
0,US90214J1016,2U INC,TWOU,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock
96,AU00000088E2,88 ENERGY LTD,POQ,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock
172,JP3160590000,BIRDMAN INC,7063,BBG00NG9S234,BBG00NG9S234,BBG00NG9S261,JP,Equity,7063,Common Stock,Common Stock
179,KYG2953R1149,AAC TECHNOLOGIES HOLDINGS IN,A2XN,BBG000C162Q4,BBG000C162Q4,BBG001SKMWN2,GR,Equity,A2XN,Common Stock,Common Stock
251,KYG000371040,AAG ENERGY HOLDINGS LTD,2686,BBG009CXGZ63,BBG009CXGZ45,BBG009CXGZ54,HK,Equity,2686,Common Stock,Common Stock
...,...,...,...,...,...,...,...,...,...,...,...
3,US9895063082,ZIEGLER COS INC/THE,ZGCO,BBG000CG8F44,BBG000CG8F44,BBG001S6VMH2,US,Equity,ZGCO,Common Stock,Common Stock
6,CA98959W2031,ZINC ONE RESOURCES INC,Z/H,BBG000RG2ZZ6,BBG000RG2ZZ6,BBG001STKXP1,CN,Equity,Z/H,Common Stock,Common Stock
80,US9897018183,ZIONS BANCORP NA,ZION V6.95 09/15/28,BBG0058QYKL7,None,None,NASDAQ/NGM,Pfd,ZION 6.95 09/15/28,PUBLIC,Preferred Stock
81,CA98977R1038,ZOOMMED INC,ZMD/H,BBG000BYKYT4,BBG000BYKYT4,BBG001SJS399,CN,Equity,ZMD/H,Common Stock,Common Stock


In [106]:
df_figi = df_figi.rename(columns = {'longName': 'longName-FIGI', 'symbol': 'symbol-FIGI', 'exchCode': 'exchCode-FIGI', 'marketSector': 'marketSector-FIGI', 'securityDescription': 'securityDescription-FIGI', 'securityType': 'securityType-FIGI', 'securityType2': 'securityType2-FIGI'})

df_figi 

,ISIN,longName-FIGI,symbol-FIGI,figi,compositeFIGI,shareClassFIGI,exchCode-FIGI,marketSector-FIGI,securityDescription-FIGI,securityType-FIGI,securityType2-FIGI
0,US90214J1016,2U INC,TWOU,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock
96,AU00000088E2,88 ENERGY LTD,POQ,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock
172,JP3160590000,BIRDMAN INC,7063,BBG00NG9S234,BBG00NG9S234,BBG00NG9S261,JP,Equity,7063,Common Stock,Common Stock
179,KYG2953R1149,AAC TECHNOLOGIES HOLDINGS IN,A2XN,BBG000C162Q4,BBG000C162Q4,BBG001SKMWN2,GR,Equity,A2XN,Common Stock,Common Stock
251,KYG000371040,AAG ENERGY HOLDINGS LTD,2686,BBG009CXGZ63,BBG009CXGZ45,BBG009CXGZ54,HK,Equity,2686,Common Stock,Common Stock
...,...,...,...,...,...,...,...,...,...,...,...
3,US9895063082,ZIEGLER COS INC/THE,ZGCO,BBG000CG8F44,BBG000CG8F44,BBG001S6VMH2,US,Equity,ZGCO,Common Stock,Common Stock
6,CA98959W2031,ZINC ONE RESOURCES INC,Z/H,BBG000RG2ZZ6,BBG000RG2ZZ6,BBG001STKXP1,CN,Equity,Z/H,Common Stock,Common Stock
80,US9897018183,ZIONS BANCORP NA,ZION V6.95 09/15/28,BBG0058QYKL7,None,None,NASDAQ/NGM,Pfd,ZION 6.95 09/15/28,PUBLIC,Preferred Stock
81,CA98977R1038,ZOOMMED INC,ZMD/H,BBG000BYKYT4,BBG000BYKYT4,BBG001SJS399,CN,Equity,ZMD/H,Common Stock,Common Stock


In [99]:
len(df['ISIN'].unique())

17511

### Merge to the dataset

In [107]:
df_withFIGI = pd.merge(df, df_figi, how='outer', on='ISIN')
df_withFIGI

,Code-EOD,Country-EOD,Currency-EOD,Exchange-EOD,ISIN,Name-EOD,Type-EOD,previousClose-EOD,previousCloseDate-EOD,country-Inv,...,longName-FIGI,symbol-FIGI,figi,compositeFIGI,shareClassFIGI,exchCode-FIGI,marketSector-FIGI,securityDescription-FIGI,securityType-FIGI,securityType2-FIGI
0,TWOU,USA,USD,US,US90214J1016,2U Inc,Common Stock,9.200,2022-08-02,united states,...,2U INC,TWOU,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock
1,88E,UK,GBX,LSE,AU00000088E2,88 Energy Ltd,Common Stock,0.625,2022-08-03,united kingdom,...,88 ENERGY LTD,POQ,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock
2,7063,Japan,JPY,TSE,JP3160590000,A Dot Co Ltd,Common Stock,2064.000,2022-08-03,japan,...,BIRDMAN INC,7063,BBG00NG9S234,BBG00NG9S234,BBG00NG9S261,JP,Equity,7063,Common Stock,Common Stock
3,2018,Hong Kong,HKD,HK,KYG2953R1149,AAC Technologies Holdings Inc,Common Stock,14.580,2022-08-03,hong kong,...,AAC TECHNOLOGIES HOLDINGS IN,A2XN,BBG000C162Q4,BBG000C162Q4,BBG001SKMWN2,GR,Equity,A2XN,Common Stock,Common Stock
4,2686,Hong Kong,HKD,HK,KYG000371040,AAG Energy Holdings Ltd,Common Stock,1.410,2022-08-03,hong kong,...,AAG ENERGY HOLDINGS LTD,2686,BBG009CXGZ63,BBG009CXGZ45,BBG009CXGZ54,HK,Equity,2686,Common Stock,Common Stock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17506,NaN,NaN,NaN,NaN,US98979J1097,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17507,NaN,NaN,NaN,NaN,CA98977K1084,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17508,NaN,NaN,NaN,NaN,CA98977R1038,NaN,NaN,NaN,NaN,NaN,...,ZOOMMED INC,ZMD/H,BBG000BYKYT4,BBG000BYKYT4,BBG001SJS399,CN,Equity,ZMD/H,Common Stock,Common Stock
17509,NaN,NaN,NaN,NaN,GB00BMHTHT14,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
df_withFIGI.to_csv('./Data/withFIGI.csv', index=False)

## Check FIGI Name

In [111]:
from fuzzywuzzy import fuzz
df_Name = df_withFIGI[['ISIN', 'Name_197', 'longName-FIGI']].dropna(subset=['longName-FIGI'])

df_Name

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,ISIN,Name_197,longName-FIGI
0,US90214J1016,2U INC,2U INC
1,AU00000088E2,88 ENERGY LTD,88 ENERGY LTD
2,JP3160590000,A DOT CO LTD,BIRDMAN INC
3,KYG2953R1149,AAC TECHNOLOGIES HOLDINGS IN,AAC TECHNOLOGIES HOLDINGS IN
4,KYG000371040,AAG ENERGY HOLDINGS LTD,AAG ENERGY HOLDINGS LTD
...,...,...,...
17501,US9895063082,ZIEGLER COS INC/THE,ZIEGLER COS INC/THE
17502,CA98959W2031,ZINC ONE RESOURCES INC,ZINC ONE RESOURCES INC
17505,US9897018183,ZIONS BANCORPORATION,ZIONS BANCORP NA
17508,CA98977R1038,ZOOMMED INC,ZOOMMED INC


In [113]:
allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['Name_197'],row['longName-FIGI'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

progress: 0.00%
progress: 9.99%
progress: 19.99%
progress: 29.98%
progress: 39.98%
progress: 49.97%
progress: 59.97%
progress: 69.96%
progress: 79.95%
progress: 99.94%


In [114]:
df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])
df_match

,ISIN,Name_197,longName-FIGI,fuzz_socre1
0,US90214J1016,2U INC,2U INC,100
1,AU00000088E2,88 ENERGY LTD,88 ENERGY LTD,100
2,JP3160590000,A DOT CO LTD,BIRDMAN INC,26
3,KYG2953R1149,AAC TECHNOLOGIES HOLDINGS IN,AAC TECHNOLOGIES HOLDINGS IN,100
4,KYG000371040,AAG ENERGY HOLDINGS LTD,AAG ENERGY HOLDINGS LTD,100
...,...,...,...,...
17501,US9895063082,ZIEGLER COS INC/THE,ZIEGLER COS INC/THE,100
17502,CA98959W2031,ZINC ONE RESOURCES INC,ZINC ONE RESOURCES INC,100
17505,US9897018183,ZIONS BANCORPORATION,ZIONS BANCORP NA,78
17508,CA98977R1038,ZOOMMED INC,ZOOMMED INC,100


In [115]:
df_match[df_match['fuzz_socre1']<80]

,ISIN,Name_197,longName-FIGI,fuzz_socre1
2,JP3160590000,A DOT CO LTD,BIRDMAN INC,26
15,SE0006256558,ABSOLENT GROUP AB,ABSOLENT AIR CARE GROUP AB,79
38,JP3108120001,ACRODEA INC,THE WHY HOW DO COMPANY INC,32
53,LU1250154413,ADO PROPERTIES SA,ADLER GROUP SA,58
57,US00486H1059,ADTRAN INC,ADTRAN HOLDINGS INC,69
...,...,...,...,...
16791,KYG735374020,SCOTTISH RE GROUP LTD PFD B 7.25%,SCOTTISH RE GROUP LTD,79
17055,US7847882005,"SYNTHETIC FIXED-INCOME SEC STRATS 2004-2, 6.50...",STRATS-FON-04-2,36
17103,MHY8564M1057,TEEKAY LNG PARTNERS LP,SEAPEAK LLC,30
17110,US8794338520,TELEPHONE AND DATA SYSTEMS INC 6.625% PFD,TELEPHONE & DATA SYSTEMS,71


In [116]:
df_match[df_match['fuzz_socre1']<60]

,ISIN,Name_197,longName-FIGI,fuzz_socre1
2,JP3160590000,A DOT CO LTD,BIRDMAN INC,26
38,JP3108120001,ACRODEA INC,THE WHY HOW DO COMPANY INC,32
53,LU1250154413,ADO PROPERTIES SA,ADLER GROUP SA,58
78,US0077371096,AEROCENTURY CORP,MEGA MATRIX CORP,25
121,JP3102000001,AISIN SEIKI CO LTD,AISIN CORP,57
...,...,...,...,...
16577,US74838C1062,QUIKSILVER INC,BOARDRIDERS INC,21
16581,US7472623013,QVC INC 6 3/8 09/13/67 PFD,QVC INC,42
16690,SE0010520155,ROLLING OPTICS HOLDING AB,ICTA AB,31
17055,US7847882005,"SYNTHETIC FIXED-INCOME SEC STRATS 2004-2, 6.50...",STRATS-FON-04-2,36
